In [1]:
# Bibliotecas
# Webscrapping
import requests as rq
import json
#from pandas.io.json import json_normalize
from pandas.io.json._normalize import json_normalize

# Tratamento e análise
import pandas as pd
import numpy as np
import scipy.stats as stats

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Seleção
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV

# Pré-processamento
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Modelos de Regressão
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
# Modelo adicional de regressão:
# from sklearn.tree import DecisionTreeRegressor

# Modelos de Classificação
from sklearn.tree import DecisionTreeClassifier
# Modelo adicional de classificação:
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier

# Pipeline
from sklearn.pipeline import make_pipeline

# Métricas dos nossos modelos
from sklearn.metrics import classification_report, accuracy_score, r2_score, mean_squared_error

### 1. Webscrapping e Preparo inicial do Dataset

Função que vai fazer o scraping dos dados da ZAP imoveis

In [2]:
#Configura o display do pandas
pd.options.display.max_columns = 99
pd.options.display.max_rows = 99

#Imita um navegador para passar restricoes
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

#Variaveis fixas
vPagFinal = 10          #Numero total de paginas a serem buscadas, se houver menos o script controla
vTransacao = "venda"    #Tipo de transacao: venda ou aluguel

#Listas para o script pegar automaticamente, pode aumentar ou diminuir com novos dados
vImovelLista = ["apartamentos"]   #"casas","casas-de-vila","casas-de-condominio"
vZonaLista = ["TODOS"] #"zona-norte","centro","zona-sul", "zona-oeste", "zona-leste", "TODOS", etc
vBairroLista = ["TODOS"]   #"barra-da-tijuca","pechincha","freguesia-jacarepagua","anil" ou "TODOS" para pesquisar todos de determinada area
vQuartos = 4          #Quantidade de quartos
vValorMin = 50000      #Valor Minimo
vValorMax = 1000000     #Valor Maximo

# https://www.zapimoveis.com.br/venda/apartamentos/ce+fortaleza/

def get_list_imoveis(vUF: str, vCidadeLista: list):
  #Para cada cidade na lista
  for vCidade in vCidadeLista:
      #para cada zona na lista de Zonas
      for vZona in vZonaLista:
          #Para cada bairro na lista de Bairros
          for vBairro in vBairroLista:
              #Para cada imovel na lista de imoveis
              for vImovel in vImovelLista:

                  #Define a pagina inicial
                  vPagina = 1
                      
                  #Print dos dados para acompanhar
                  print(vCidade + ' -> ' + vZona + ' -> ' + vBairro + ' -> ' + vImovel + ' -> ' + str(vQuartos))
                  
                  #Teste se o Bairro é unico, alguns ou todos
                  if vBairro == 'TODOS':
                      if vZona == 'TODOS':
                          if vCidade == 'TODOS':
                              vURL_B = "https://www.zapimoveis.com.br/" + vTransacao + "/" + vImovel + "/" + str(vQuartos) + "-Quartos"
                          else:
                              vURL_B = "https://www.zapimoveis.com.br/" + vTransacao + "/" + vImovel + "/" + vUF + "+" + vCidade
                      else:
                          vURL_B = "https://www.zapimoveis.com.br/" + vTransacao + "/" + vImovel + "/" + vUF + "+" + vCidade + "+" + vZona + "/" + str(vQuartos) + "-Quartos"
                  else:
                      vURL_B = "https://www.zapimoveis.com.br/" + vTransacao + "/" + vImovel + "/" + vUF + "+" + vCidade + "+" + vZona + "+" + vBairro + "/" + str(vQuartos) + "-Quartos"
                      
                  #Cria o dicionario
                  dfs = {}
                      
                  #Itera entre as paginas
                  while vPagina <= vPagFinal:
                          
                      #URL
                      vURL = vURL_B + "/?pagina=" + str(vPagina) + "&precoMinimo=" + str(vValorMin) + "&precoMaximo=" + str(vValorMax)
                      
                      # print('Pagina: ' + str(vPagina))
                      #print(vURL)
                      #Testa o codigo de retorno do site
                      # print(vURL+'\n')
                      vResp = rq.get(vURL,headers=headers)
                      vStat = vResp.status_code
                          
                      #Se codigo 200, entao vai adiante
                      if vStat == 200:
                          vHTML = vResp.text
                          vHTML = str(vHTML)
                              
                          #Valida se a pagina existe ou nao
                          vValPag = 'OK' if 'Não encontramos resultados' in vHTML else 'NOK'
                              
                          #Continua se a pagina existir
                          if vValPag == 'NOK':
                          
                              #Pega apenas a parte do Json do codigo fonte
                              vHTML = vHTML.split('"results":{"listings":[',1)[1]
                              vHTML = vHTML.split('],"nearbyListings":[]',1)[0]
                              vHTML = vHTML.split(',"type":"nearby"}]',1)[0]                    
                              
                              #Ao dar erro de delimitador, adicionar ou retirar chaves antes do colchetes na variavel abaixo
                              #Valida o fim da string, pois para alguns casos vem com char a menos
                              if vHTML[-2:] == '}}':
                                  v1 = '{"listings":[' + vHTML + ']}'
                              elif vHTML[-10:] == '"premium"}':
                                  vHTML = vHTML.split(',"type":"premium"}',1)[0]
                                  #vHTML = vHTML.split(vHTML[-18],1)[0]
                                  v1 = '{"listings":[' + vHTML + '}]}'
                              else:
                                  v1 = '{"listings":[' + vHTML + '}]}'
                                  
                              #Retira a marcacao de moeda, deixando apenas o valor
                              v1 = v1.replace('R$ ','')
                              v1 = v1.replace(',"superPremiumListings":[}]}','}')
                              v1 = v1.split(';(function',1)[0]
                              v1 = v1.split(',"advertisers"',1)[0]
                              
                              # print(v1)
                              j = json.loads(v1)
                                
                              #Cria o dataframe do pandas, já normalizando o json
                              df = json_normalize(j['listings'])
                              
                              #Deixa somente as colunas utilizaveis
                              df = df[['type','link.href','account.name','listing.usableAreas','listing.totalAreas','listing.title','listing.description','listing.createdAt','listing.updatedAt','listing.floors',
                                        'listing.parkingSpaces','listing.address.zipCode','listing.address.street','listing.address.neighborhood','listing.address.streetNumber','listing.suites',
                                        'listing.bathrooms','listing.bedrooms','listing.advertiserContact.phones','listing.whatsappNumber','listing.pricingInfo.salePrice','listing.pricingInfo.yearlyIptu',
                                        'listing.pricingInfo.monthlyCondoFee','listing.publicationType','listing.unitTypes','listing.unitSubTypes','listing.usageTypes','listing.amenities']]
                                  
                              #Insere a coluna com o tipo de imovel
                              df['imvl_type'] = vImovel
                                  
                              #Tratamento dos dados
                              df['listing.publicationType'] = df['listing.publicationType'].fillna('Standard')
                                  
                              #Retira o colchetes, transformando a lista em string dentro da coluna
                              df['listing.floors'] = [''.join(map(str, l)) for l in df['listing.floors']]
                              df['listing.unitTypes'] = [''.join(map(str, l)) for l in df['listing.unitTypes']]
                              df['listing.unitSubTypes'] = ['|'.join(map(str, l)) for l in df['listing.unitSubTypes']]
                              df['listing.parkingSpaces'] = [''.join(map(str, l)) for l in df['listing.parkingSpaces']]
                              df['listing.suites'] = [''.join(map(str, l)) for l in df['listing.suites']]
                              df['listing.bathrooms'] = [''.join(map(str, l)) for l in df['listing.bathrooms']]
                              df['listing.usageTypes'] = ['|'.join(map(str, l)) for l in df['listing.usageTypes']]
                              df['listing.totalAreas'] = [''.join(map(str, l)) for l in df['listing.totalAreas']]
                              df['listing.bedrooms'] = [''.join(map(str, l)) for l in df['listing.bedrooms']]
                              df['listing.amenities'] = ['|'.join(map(str, l)) for l in df['listing.amenities']]
                              df['listing.usableAreas'] = [''.join(map(str, l)) for l in df['listing.usableAreas']]
                                  
                              #Cria colunas baseadas na coluna listing.amenities
                              df['listing.pool'] = df['listing.amenities'].map(lambda x: 'True' if 'POOL' in x else 'False')                  #Piscina sim ou nao
                              df['listing.sauna'] = df['listing.amenities'].map(lambda x: 'True' if 'SAUNA' in x else 'False')                #Sauna sim ou nao
                              df['listing.backyard'] = df['listing.amenities'].map(lambda x: 'True' if 'BACKYARD' in x else 'False')          #Quintal sim ou nao
                              df['listing.garden'] = df['listing.amenities'].map(lambda x: 'True' if 'GARDEN' in x else 'False')              #Jardim sim ou nao
                              df['listing.barbgrill'] = df['listing.amenities'].map(lambda x: 'True' if 'BARBECUE_GRILL' in x else 'False')   #Churrasqueira sim ou nao
                              df['listing.partyhall'] = df['listing.amenities'].map(lambda x: 'True' if 'PARTY_HALL' in x else 'False')       #Salao de festas sim ou nao
                              df['listing.tenniscourt'] = df['listing.amenities'].map(lambda x: 'True' if 'TENNIS_COURT' in x else 'False')   #Quadra de Tennis sim ou nao
                              df['listing.sportcourt'] = df['listing.amenities'].map(lambda x: 'True' if 'SPORTS_COURT' in x else 'False')    #Quadra de Esportes sim ou nao
                              df['listing.bathtub'] = df['listing.amenities'].map(lambda x: 'True' if 'BATHTUB' in x else 'False')            #Banheira sim ou nao
                              df['listing.soundproofing'] = df['listing.amenities'].map(lambda x: 'True' if 'SOUNDPROOFING' in x else 'False')#Prova de som sim ou nao
                              df['listing.fireplace'] = df['listing.amenities'].map(lambda x: 'True' if 'FIREPLACE' in x else 'False')        #Lareira sim ou nao
                              df['listing.gym'] = df['listing.amenities'].map(lambda x: 'True' if 'GYM' in x else 'False')                    #Academia sim ou nao
                              df['listing.hottub'] = df['listing.amenities'].map(lambda x: 'True' if 'HOT_TUB' in x else 'False')             #Hidromassagem sim ou nao
                              df['listing.furnished'] = df['listing.amenities'].map(lambda x: 'True' if 'FURNISHED' in x else 'False')        #Mobiliado sim ou nao
                              df['listing.guestpark'] = df['listing.amenities'].map(lambda x: 'True' if 'GUEST_PARKING' in x else 'False')    #Estacionamento Visitantes sim ou nao
                              df['listing.playground'] = df['listing.amenities'].map(lambda x: 'True' if 'PLAYGROUND' in x else 'False')      #Playground sim ou nao
                              df['listing.mountainview'] = df['listing.amenities'].map(lambda x: 'True' if 'MOUNTAIN_VIEW' in x else 'False')    #Vista da montanha sim ou nao
                                  
                              #Cria a entrada variavel no dicionario
                              dfs['df_' + str(vPagina)] = df
                  
                              #Incrementa um na pagina
                              vPagina = vPagina + 1
                              
                          else:
                              break
                          
                      #Sai do Loop
                      else:
                          print(vURL)
                          print('\n')
                          print(vStat)
                          break
                          
                  #Cria a lista 
                  vAcaoFimLista = []
                      
                  #Para cada entrada dinamica criada no Dicionário, adiciona na lista
                  for i in dfs.keys(): 
                      #print(i)
                      vAcaoFimLista.append(dfs[i])
                      #df_acoes = pandas.DataFrame().append(dfs[i], ignore_index=False)
                      
                  #Concatena os dados da lista em um unico dataframe
                  df_Zap = pd.concat(vAcaoFimLista, sort=False)
                  
                  cols_drop = ['type', 'link.href', 'listing.floors','listing.title', 'listing.address.street', 'account.name', 'listing.createdAt', 'listing.updatedAt', 'listing.floors', 
                              'listing.description', 'listing.address.zipCode', 'listing.amenities', 'listing.address.neighborhood',
                              'listing.address.streetNumber', 'listing.advertiserContact.phones', 'listing.whatsappNumber', 
                              'listing.publicationType', 'listing.mountainview', 'listing.soundproofing',
                               'listing.totalAreas', 'listing.unitSubTypes', 'listing.unitTypes', 'listing.usageTypes']
                  df_Zap = df_Zap.drop(cols_drop, axis=1)
                  df_Zap['listing.address.state'] = vUF
                  df_sorted = df_Zap.sort_index(axis=1)

                  return df_sorted

In [3]:
def scraping():
    locais = [['rn', 'natal'], ['pe', 'recife'], ['ba', 'salvador'],
              ['pa', 'belem'], ['am', 'manaus'], ['to', 'palmas'],
              ['mt', 'cuiaba'],['go', 'goiania'],['ms', 'campo-grande'],
              ['pr', 'curitiba'],['sc', 'florianopolis'], ['rs', 'porto-alegre']]

    print("Iniciando Scrapping: ")
    dataframes = []
    for estado, capital in locais:
        dataframes.append(get_list_imoveis(estado, [capital]))
    print("Finalizou.")
    return dataframes
#dataframes = scraping()
#df_web_scrapping = pd.concat(dataframes, ignore_index=True)

In [4]:
df_web_scrapping = pd.read_csv("data_scrapping.csv", dtype = str) #str para pegar do jeito o scraping botou
df_web_scrapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11565 entries, 0 to 11564
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            11565 non-null  object
 1   listing.address.state                11565 non-null  object
 2   listing.backyard                     11565 non-null  object
 3   listing.barbgrill                    11565 non-null  object
 4   listing.bathrooms                    11565 non-null  object
 5   listing.bathtub                      11565 non-null  object
 6   listing.bedrooms                     11565 non-null  object
 7   listing.fireplace                    11565 non-null  object
 8   listing.furnished                    11565 non-null  object
 9   listing.garden                       11565 non-null  object
 10  listing.guestpark                    11565 non-null  object
 11  listing.gym                          1156

In [5]:
df_web_scrapping_copy = df_web_scrapping.replace([np.nan, ''], "normal")

colunas_numericas = ['listing.bathrooms', 'listing.pricingInfo.salePrice', 'listing.usableAreas',
                     'listing.pricingInfo.salePrice', 'listing.pricingInfo.monthlyCondoFee']
# retirei o monthlyCondoFee, suites e parkingSpaces por que vou tratá-los quando tudo tiver junto

colunas_bool = ['listing.backyard', 'listing.barbgrill', 'listing.bathtub',
                'listing.fireplace', 'listing.furnished', 'listing.garden',
                'listing.guestpark', 'listing.gym', 'listing.hottub',
                'listing.partyhall', 'listing.playground', 'listing.pool',
                'listing.sauna', 'listing.sportcourt', 'listing.tenniscourt']

df_web_scrapping_copy[colunas_bool] = df_web_scrapping_copy[colunas_bool].astype(bool)
# mudando os tipos das colunas para ficar condizente


# encontrei o erro do 1000 ser entendido como 1.000, resolvendo aqui
for coluna in colunas_numericas:
    df_web_scrapping_copy[coluna] = df_web_scrapping_copy[coluna].str.replace('.', '') 
    # tirando os pontos pra converter certinho

df_web_scrapping_copy['listing.pricingInfo.salePrice'] = df_web_scrapping_copy['listing.pricingInfo.salePrice'].replace(['normal', ''], np.nan)
df_web_scrapping_copy = df_web_scrapping_copy.dropna()

df_web_scrapping_copy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11544 entries, 0 to 11564
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            11544 non-null  object
 1   listing.address.state                11544 non-null  object
 2   listing.backyard                     11544 non-null  bool  
 3   listing.barbgrill                    11544 non-null  bool  
 4   listing.bathrooms                    11544 non-null  object
 5   listing.bathtub                      11544 non-null  bool  
 6   listing.bedrooms                     11544 non-null  object
 7   listing.fireplace                    11544 non-null  bool  
 8   listing.furnished                    11544 non-null  bool  
 9   listing.garden                       11544 non-null  bool  
 10  listing.guestpark                    11544 non-null  bool  
 11  listing.gym                          11544 non

In [6]:
# Pegando dados vindos do Kaggle para juntar aos do Scraping
df_imoveis_kaggle = pd.read_csv('./dataZAP.csv', encoding='utf-8', engine='python', sep=";", thousands='.', decimal=',')
# Selecionando os dados de venda
df_imoveis_venda_kaggle = df_imoveis_kaggle[df_imoveis_kaggle['listing.pricingInfo.isSale'] == True]

df_imoveis_venda_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9553 entries, 25 to 35755
Data columns (total 78 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   account.licenseNumber                 9553 non-null   object
 1   account.name                          9553 non-null   object
 2   imvl_type                             9553 non-null   object
 3   listing.acceptExchange                9553 non-null   bool  
 4   listing.address.city                  9553 non-null   object
 5   listing.address.confidence            9553 non-null   object
 6   listing.address.country               9553 non-null   object
 7   listing.address.level                 9553 non-null   object
 8   listing.address.neighborhood          9553 non-null   object
 9   listing.address.point.lat             9553 non-null   object
 10  listing.address.point.lon             9553 non-null   object
 11  listing.address.point.source     

In [7]:
cols_drop = ['listing.floors', 'listing.legacyId', 'listing.amenities', 'listing.address.street',  'listing.publicationType',
              'listing.pricingInfo.period', 'listing.unitSubTypes', 'listing.acceptExchange', 'listing.pricingInfo.rentalPrice',
              'listing.pricingInfo.rentalTotalPrice', 'account.licenseNumber', 'account.name', 'listing.address.country', 'listing.address.point.lat', 'listing.address.level',
              'listing.address.point.lon', 'listing.updatedAt', 'listing.address.point.source', 'listing.address.precision', 'listing.displayAddressType', 'listing.pricingInfo.businessLabel',
              'listing.address.confidence', 'listing.propertyType', 'type', 'listing.usageTypes', 'listing.advertiserId', 'listing.description', 'listing.totalAreas', 'listing.pricingInfo.price',
              'listing.address.streetNumber', 'listing.address.zipCode', 'listing.businessTypeContext', 'listing.title', 'listing.soundproofing', 'listing.address.city',
              'listing.createdAt', 'listing.pricingInfo.businessType', 'listing.portal', 'listing.unitsOnTheFloor', 'listing.unitFloor', 'listing.address.zone',
              'listing.listingType', 'listing.externalId', 'listing.preview', 'listing.id', 'listing.isInactive', 'listing.subtitle', 'listing.unitTypes',
              'listing.link', 'listing.mountainview', 'listing.pricingInfo.isRent', 'listing.address.neighborhood', 'listing.pricingInfo.isSale']


# Basicamente tirando as colunas que não aparecem no scraping
df_imoveis_venda_kaggle = df_imoveis_venda_kaggle.drop(cols_drop, axis=1)

print(df_imoveis_venda_kaggle['listing.pricingInfo.salePrice'].value_counts())

# Deletar os dados que tinham MUITOS valores vazios, os dados repetidos e aqueles que não influenciam no valor dos imoveis  

df_imoveis_venda_kaggle.info()

listing.pricingInfo.salePrice
3500000    173
1800000    168
1200000    156
2500000    147
1500000    146
          ... 
1570000      1
915000       1
657574       1
845000       1
2890000      1
Name: count, Length: 853, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 9553 entries, 25 to 35755
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            9553 non-null   object
 1   listing.address.state                9553 non-null   object
 2   listing.backyard                     9553 non-null   bool  
 3   listing.barbgrill                    9553 non-null   bool  
 4   listing.bathrooms                    9553 non-null   object
 5   listing.bathtub                      9553 non-null   bool  
 6   listing.bedrooms                     9553 non-null   int64 
 7   listing.fireplace                    9553 non-null   bool  
 8   listing

In [8]:
colunas_numericas = ['listing.bathrooms', 'listing.parkingSpaces',
              'listing.suites', 'listing.pricingInfo.salePrice',
              'listing.pricingInfo.monthlyCondoFee', 'listing.usableAreas']

# Unindo os dois dados

df_vendas_final = pd.concat([df_imoveis_venda_kaggle, df_web_scrapping_copy], ignore_index=True)

# aqui vou substituir os normals que tem pelo valor
# suites e parkingSpace coloquei 1 pq a maioria é 1

df_vendas_final['listing.suites'] = df_vendas_final['listing.suites'].replace('normal', 1)
df_vendas_final['listing.parkingSpaces'] = df_vendas_final['listing.parkingSpaces'].replace('normal', 1)

# monthlyCondoFee coloquei a média pois é um valor numérico mais abrangente, com poucas repetições
mediaCF = int(df_vendas_final.loc[df_vendas_final['listing.pricingInfo.monthlyCondoFee']\
                                    != 'normal', 'listing.pricingInfo.monthlyCondoFee'].astype(float).mean())
df_vendas_final['listing.pricingInfo.monthlyCondoFee'] = df_vendas_final['listing.pricingInfo.monthlyCondoFee'].replace(['normal', '', np.nan], mediaCF) 


df_vendas_final[colunas_numericas] = df_vendas_final[colunas_numericas].astype(int)

df_vendas_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21097 entries, 0 to 21096
Data columns (total 25 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            21097 non-null  object
 1   listing.address.state                21097 non-null  object
 2   listing.backyard                     21097 non-null  bool  
 3   listing.barbgrill                    21097 non-null  bool  
 4   listing.bathrooms                    21097 non-null  int64 
 5   listing.bathtub                      21097 non-null  bool  
 6   listing.bedrooms                     21097 non-null  object
 7   listing.fireplace                    21097 non-null  bool  
 8   listing.furnished                    21097 non-null  bool  
 9   listing.garden                       21097 non-null  bool  
 10  listing.guestpark                    21097 non-null  bool  
 11  listing.gym                          2109

In [9]:
#try:
#    df_vendas_final[colunas_numericas] = df_vendas_final[colunas_numericas].astype(int)
#except ValueError:
#    df_vendas_final[colunas_numericas] = df_vendas_final[colunas_numericas].apply(pd.to_numeric, errors='coerce')

In [10]:
def Regiao(df_imoveis_venda):
    dicionario = {'Acre': 'Norte', 'Amazonas': 'Norte', 'Rondônia': 'Norte', 'Pará': 'Norte', 'Tocantins': 'Norte', 
                  'Maranhão': 'Nordeste', 'Piauí': 'Nordeste', 'Bahia': 'Nordeste', 'Ceará': 'Nordeste', 'Rio Grande do Norte': 'Nordeste', 
                  'Paraíba': 'Nordeste', 'Pernambuco': 'Nordeste', 'Alagoas': 'Nordeste', 'Sergipe': 'Nordeste', 
                  'Minas Gerais': 'Sudeste', 'São Paulo': 'Sudeste', 'Rio de Janeiro': 'Sudeste', 'Espírito Santo': 'Sudeste', 
                  'Paraná': 'Sul', 'Santa Catarina': 'Sul', 'Rio Grande do Sul': 'Sul', 
                  'Mato Grosso': 'Centro-Oeste', 'Mato Grosso do Sul': 'Centro-Oeste', 'Goiás': 'Centro-Oeste', 'Distrito Federal': 'Centro-Oeste',
                  'am': 'Norte', 'pa': 'Norte', 'ba': 'Nordeste', 'pe': 'Nordeste', 'ce': 'Nordeste', 'rn': 'Nordeste', 'mt': 'Centro-Oeste',
                  'go': 'Centro-Oeste', 'ms': 'Centro-Oeste', 'pr': 'Sul', 'sc': 'Sul', 'rs': 'Sul', 'to': 'Norte'}
    for nome in dicionario:
        df_imoveis_venda['listing.address.state'] = df_imoveis_venda['listing.address.state'].replace(nome, dicionario[nome])
    return df_imoveis_venda['listing.address.state']

In [11]:
Regiao(df_vendas_final)
# Organizando os dados por região do Brasil

df_vendas_final['listing.address.state'].value_counts()
print(df_vendas_final['listing.address.state'].value_counts())

listing.address.state
Sudeste         8861
Sul             3378
Nordeste        3275
Centro-Oeste    3010
Norte           2573
Name: count, dtype: int64


In [12]:
df_vendas_final = df_vendas_final.replace("normal", np.nan)

# yearlyIPTU tem muitos vazios, melhor tirar

df_vendas_final = df_vendas_final.drop("listing.pricingInfo.yearlyIptu", axis = 1)
df_vendas_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21097 entries, 0 to 21096
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            21097 non-null  object
 1   listing.address.state                21097 non-null  object
 2   listing.backyard                     21097 non-null  bool  
 3   listing.barbgrill                    21097 non-null  bool  
 4   listing.bathrooms                    21097 non-null  int64 
 5   listing.bathtub                      21097 non-null  bool  
 6   listing.bedrooms                     21097 non-null  object
 7   listing.fireplace                    21097 non-null  bool  
 8   listing.furnished                    21097 non-null  bool  
 9   listing.garden                       21097 non-null  bool  
 10  listing.guestpark                    21097 non-null  bool  
 11  listing.gym                          2109

In [13]:
num_nan = df_vendas_final.isna().sum()
print("Quantidade de valores NaN na coluna:\n", num_nan)

Quantidade de valores NaN na coluna:
 imvl_type                              0
listing.address.state                  0
listing.backyard                       0
listing.barbgrill                      0
listing.bathrooms                      0
listing.bathtub                        0
listing.bedrooms                       0
listing.fireplace                      0
listing.furnished                      0
listing.garden                         0
listing.guestpark                      0
listing.gym                            0
listing.hottub                         0
listing.parkingSpaces                  0
listing.partyhall                      0
listing.playground                     0
listing.pool                           0
listing.pricingInfo.monthlyCondoFee    0
listing.pricingInfo.salePrice          0
listing.sauna                          0
listing.sportcourt                     0
listing.suites                         0
listing.tenniscourt                    0
listing.usableAreas

In [14]:
df_vendas_final = df_vendas_final.drop_duplicates()

df_vendas_final[colunas_bool] = df_vendas_final[colunas_bool].astype(int) # colocando para numérico

df_vendas_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16792 entries, 0 to 21096
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   imvl_type                            16792 non-null  object
 1   listing.address.state                16792 non-null  object
 2   listing.backyard                     16792 non-null  int64 
 3   listing.barbgrill                    16792 non-null  int64 
 4   listing.bathrooms                    16792 non-null  int64 
 5   listing.bathtub                      16792 non-null  int64 
 6   listing.bedrooms                     16792 non-null  object
 7   listing.fireplace                    16792 non-null  int64 
 8   listing.furnished                    16792 non-null  int64 
 9   listing.garden                       16792 non-null  int64 
 10  listing.guestpark                    16792 non-null  int64 
 11  listing.gym                          16792 non

In [15]:
df_vendas_final['listing.bedrooms'] = df_vendas_final['listing.bedrooms'].astype(int)
df_vendas_final['listing.backyard'] = df_vendas_final['listing.backyard'].astype(int)

# Mudando algumas colunas que foram mudadas antes mas que voltaram a ser object por erro na junção dos datasets

In [16]:
# Reordenando as colunas

colunas_bool_x = ['listing.backyard', 'listing.barbgrill', 'listing.bathtub',
                'listing.fireplace', 'listing.furnished', 'listing.garden',
                'listing.guestpark', 'listing.gym', 'listing.hottub',
                'listing.partyhall', 'listing.playground', 'listing.pool',
                'listing.sauna', 'listing.sportcourt', 'listing.tenniscourt']

colunas_numericas_x = ['listing.bathrooms', 'listing.parkingSpaces',
                     'listing.suites', 'listing.pricingInfo.monthlyCondoFee', 
                     'listing.usableAreas', 'listing.bedrooms']

colunas_multcategoricas_x = ['imvl_type', 'listing.address.state']

coluna_y = ['listing.pricingInfo.salePrice']

nova_ordem = colunas_numericas_x + colunas_bool_x + colunas_multcategoricas_x + coluna_y

df_reordered = df_vendas_final.copy()

df_reordered = df_reordered.reindex(columns=nova_ordem)

In [19]:
# Mudar os noms das colunas para retirar o listing.

df_reordered.columns = df_reordered.columns.str.replace('listing.', '')
df_reordered.columns = df_reordered.columns.str.replace('pricingInfo.', '')
df_reordered.columns = df_reordered.columns.str.replace('state', 'region') 
# mudar para fazer sentido com o nome

df_reordered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16792 entries, 0 to 21096
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   bathrooms        16792 non-null  int64 
 1   parkingSpaces    16792 non-null  int64 
 2   suites           16792 non-null  int64 
 3   monthlyCondoFee  16792 non-null  int64 
 4   usableAreas      16792 non-null  int64 
 5   bedrooms         16792 non-null  int64 
 6   backyard         16792 non-null  int64 
 7   barbgrill        16792 non-null  int64 
 8   bathtub          16792 non-null  int64 
 9   fireplace        16792 non-null  int64 
 10  furnished        16792 non-null  int64 
 11  garden           16792 non-null  int64 
 12  guestpark        16792 non-null  int64 
 13  gym              16792 non-null  int64 
 14  hottub           16792 non-null  int64 
 15  partyhall        16792 non-null  int64 
 16  playground       16792 non-null  int64 
 17  pool             16792 non-null  int

In [20]:
df_reordered.to_csv("df_vendas_final.csv", index = False) # salvando

print("Salvou.")

Salvou.
